<a href="https://colab.research.google.com/github/rahiakela/fluent-python-book-practice/blob/master/part-v-control-flow/14_iterables_iterators_and_generators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Iterables, iterators and generators

Iteration is fundamental to data processing. And when scanning datasets that don’t fit in memory, we need a way to fetch the items lazily, that is, one at a time and on demand. This is what the Iterator pattern is about.

Python does not have macros like Lisp, so abstracting away the Iterator pattern required changing the language: the yield keyword was added
in Python 2.2 (2001). The yield keyword allows the construction of generators, which work as iterators.

Python 3 uses generators in many places. Even the range() built-in now returns a generator-like object instead of full-blown lists like before. If you must build a list from range, you have to be explicit, e.g. list(range(100)).

Every collection in Python is iterable, and iterators are used internally to support:

- for loops;
- collection types construction and extension;
- looping over text files line by line;
- list, dict and set comprehensions;
- tuple unpacking;
- unpacking actual parameters with * in function calls.

## Sentence take #1: a sequence of words

We’ll start our exploration of iterables by implementing a Sentence class: you give its constructor a string with some text, and then you can iterate word by word.

In [ ]:
import re
import reprlib

In [ ]:
RE_WORD = re.compile("\w+")

class Sentence:

  def __init__(self, text):
    self.text = text
    # returns a list with all non-overlapping matches of the regular expression, as a list of strings.
    self.words = RE_WORD.findall(text)

  def __getitem__(self, index):
    # self.words holds the result of .findall, so we simply return the word at the given index.
    return self.words[index]

  # To complete the sequence protocol, we implement __len__ — but it is not needed to make an iterable object.
  def __len__(self):
    return len(self.words)

  def __repr__(self):
    # generate abbreviated string representations of data structures that can be very large
    return "Sentence(%s)" % reprlib.repr(self.text)

By default, reprlib.repr limits the generated string to 30 characters.

In [ ]:
s = Sentence('"The time has come," the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [ ]:
# Sentence instances are iterable
for word in s:
  print(word)

The
time
has
come
the
Walrus
said


In [ ]:
# Being iterable, Sentence objects can be used as input to build lists and other iterable types.
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

In [ ]:
# because it’s also a sequence, so you can get words by index
s[0]

'The'

In [ ]:
s[5]

'Walrus'

In [ ]:
s[-1]

'said'

In [ ]:
s[-2]

'Walrus'

### Why sequences are iterable: the iter function

Every Python programmer knows that sequences are iterable. Now we’ll see precisely why.

Whenever the interpreter needs to iterate over an object x, it automatically calls iter(x).

The iter built-in function:

- Checks whether the object implements, __iter__, and calls that to obtain an iterator;
- If __iter__ is not implemented, but __getitem__ is implemented, Python creates an iterator that attempts to fetch items in order, starting from index 0 (zero);
- If that fails, Python raises TypeError, usually saying "'C' object is not iterable", where C is the class of the target object.

That is why any Python sequence is iterable: they all implement `__getitem__`. In fact, the standard sequences also implement `__iter__`, and yours should too, because the special handling of `__getitem__` exists for backward compatibility reasons and may be gone in the future.

This is an extreme form of duck typing: an object is considered iterable not only when it implements the special method `__iter__`, but also when it implements `__getitem__`, as long as `__getitem__` accepts
int keys starting from 0.

In the goose-typing approach, the definition for an iterable is simpler but not as flexible: an object is considered iterable if it implements the `__iter__` method. No subclassing or registration is required, because abc.Iterable implements the `__subclasshook__`.

In [ ]:
class Foo:
  def __iter__(self):
    pass

In [ ]:
from collections import abc

In [ ]:
issubclass(Foo, abc.Iterable)

True

In [ ]:
f = Foo()
isinstance(f, abc.Iterable)

True

However, note that our initial Sentence class does not pass the issubclass(Sentence, abc.Iterable) test, even though it is iterable in practice.

### Iterables versus iterators

It’s important to be clear about the relationship between iterables and iterators: Python obtains iterators from iterables.

Here is a simple for loop iterating over a str. The str 'ABC' is the iterable here. You don’t see it, but there is an iterator behind the curtain:

In [ ]:
s = 'ABC'
for char in s:
  print(char)

A
B
C


If there was no for statement and we had to emulate the for machinery by hand with a while loop, this is what we’d have to write:

In [ ]:
s = 'ABC'
# Build an iterator it from the iterable.
it = iter(s)
while True:
  try:
    # Repeatedly call next on the iterator to obtain the next item.
    print(next(it))
  except StopIteration:  # The iterator raises StopIteration when there are no further items.
    # Release reference to it — the iterator object is discarded.
    del it
    break

A
B
C


StopIteration signals that the iterator is exhausted. This exception is handled internally in for loops and other iteration contexts like list comprehensions, tuple unpacking etc.

The standard interface for an iterator has two methods:

- `__next__`: Returns the next available item, raising StopIteration when there are no more items.
- `__iter__`: Returns self; this allows iterators to be used where an iterable is expected, for example, in a for loop.

In [ ]:
s3 = Sentence("Pig and Pepper")

# Obtain an iterator from s3.
it = iter(s3)

In [ ]:
# next(it) fetches the next word.
next(it)

'Pig'

In [ ]:
next(it)

'and'

In [ ]:
next(it)

'Pepper'

In [ ]:
# There are no more words, so the iterator raises a StopIteration exception.
next(it)

StopIteration: ignored

In [ ]:
# Once exhausted, an iterator becomes useless.
list(it)

[]

In [ ]:
# To go over the sentence again, a new iterator must be built.
list(iter(s3))

['Pig', 'and', 'Pepper']

Since the only methods required of an iterator are `__next__` and `__iter__`, there is no way to check whether there are remaining items, other than call next() and catch StopInteration. 

Also, it’s not possible to “reset” an iterator. If you need to start over, you need to call iter(…) on the iterable that built the iterator in the first place. 

Calling `iter(…)` on the iterator itself won’t help, because — as mentioned — Iterator.`__iter__` is implemented by returning self, so this will not reset a depleted iterator.

## Sentence take #2: a classic iterator

The next an implementation of a Sentence that is iterable because it implements
the `__iter__` special method which builds and returns a SentenceIterator. This
is how the Iterator design pattern is described in the original Design Patterns book.

We are doing it this way here just to make clear the crucial distinction between an iterable
and an iterator and how they are connected.

In [ ]:
RE_WORD = re.compile("\w+")

class Sentence:

  def __init__(self, text):
    self.text = text
    # returns a list with all non-overlapping matches of the regular expression, as a list of strings.
    self.words = RE_WORD.findall(text)

  def __repr__(self):
    # generate abbreviated string representations of data structures that can be very large
    return "Sentence(%s)" % reprlib.repr(self.text)

  """
  The __iter__ method is the only addition to the previous Sentence
  implementation. This version has no __getitem__, to make it clear that the class
  is iterable because it implements __iter__.
  """
  def __iter__(self):
    # __iter__ fulfills the iterable protocol by instantiating and returning an iterator.
    return SentenceIterator(self.words)


class SentenceIterator:

  def __init__(self, words):
    # SentenceIterator holds a reference to the list of words.
    self.words = words
    # self.index is used to determine the next word to fetch.
    self.index = 0

  def __next__(self):
    try:
      word = self.words[self.index]
    except IndexError:
      # If there is no word at self.index, raise StopIteration.
      raise StopIteration()
    self.index += 1  # Increment self.index.
    return word

  def __iter__(self):
    return self

Note that implementing `__iter__` in SentenceIterator is not actually needed for this example to work, but the it’s the right thing to do: iterators are supposed to implement both `__next__` and `__iter__`, and doing so makes our iterator pass the issubclass(SentenceInterator, abc.Iterator) test. If we had subclassed SentenceIterator from abc.Iterator we’d inherit the concrete abc.`Iterator.__iter__` method.

### Making Sentence an iterator: bad idea

A common cause of errors in building iterables and iterators is to confuse the two. To be clear: iterables have a `__iter__` method that instantiates a new iterator every time. Iterators implement a `__next__` method that returns individual items, and a `__iter__` method that returns self.

Therefore, iterators are also iterable, but iterables are not iterators.

To “support multiple traversals” it must be possible to obtain multiple independent iterators from the same iterable instance, and each iterator must keep its own internal state, so a proper implementation of the pattern requires each call to iter(my_iterable) to create a new, independent, iterator.

## Sentence take #3: a generator function

A Pythonic implementation of the same functionality uses a generator function to replace the SequenceIterator class.

In [ ]:
RE_WORD = re.compile("\w+")

class Sentence:

  def __init__(self, text):
    self.text = text
    # returns a list with all non-overlapping matches of the regular expression, as a list of strings.
    self.words = RE_WORD.findall(text)

  def __repr__(self):
    # generate abbreviated string representations of data structures that can be very large
    return "Sentence(%s)" % reprlib.repr(self.text)

  """
  The __iter__ method is the only addition to the previous Sentence
  implementation. This version has no __getitem__, to make it clear that the class
  is iterable because it implements __iter__.
  """
  def __iter__(self):
    # Iterate over self.word.
    for word in self.words:
      yield word   # Yield the current word.
    return

Now the iterator is in fact a generator object, built automatically when the `__iter__` method is called, because `__iter__` here is a generator function.

### How a generator function works

Any Python function that has the yield keyword in its body is a generator function: a function which, when called, returns a generator object. In other words, a generator function is a generator factory.

Here is the simplest function useful to demonstrate the behavior of a generator:

In [ ]:
def gen_123():
  yield 1
  yield 2
  yield 3

In [ ]:
gen_123

<function __main__.gen_123>

In [ ]:
gen_123()

<generator object gen_123 at 0x7f2e275997d8>

In [ ]:
for i in gen_123():
  print(i)

1
2
3


In [ ]:
g = gen_123()

In [ ]:
next(g)

1

In [ ]:
next(g)

2

In [ ]:
next(g)

3

In [ ]:
# When the body of the function completes, the generator object raises a StopIteration.
next(g)

StopIteration: ignored

A generator function builds a generator object which wraps the body of the function. When we invoke next(…) on the generator object, execution advances to the next yield in the function body, and the next(…) call evaluates to the value yielded when the function body is suspended. Finally, when the function body returns, the enclosing generator object raises StopIteration, in accordance with the Iterator protocol.

> Calling a generator function returns a generator.
A generator yields or produces values. A generator doesn’t
“return” values in the usual way: the return statement in the body
of a generator function causes StopIteration to be raised by the
generator object.

A generator function which prints messages when it runs.

In [ ]:
def gen_AB():
  print("start")
  yield "A"
  print("continue")
  yield "B"
  print("end.")

In [ ]:
for c in gen_AB():
  print("-->", c)

start
--> A
continue
--> B
end.


Now hopefully it’s clear how Sentence.`__iter__` works: `__iter__` is
generator function which, when called, builds a generator object which implements the iterator interface, so the SentenceIterator class is no longer needed.

This second version of Sentence is much shorter than the first, but it’s not as lazy as it could be. 

Nowadays, laziness is considered a good trait, at least in programming languages
and APIs. A lazy implementation postpones producing values to the last possible
moment. This saves memory and may avoid useless processing as well.

## Sentence take #4: a lazy implementation

The Iterator interface is designed to be lazy: next(my_iterator) produces one item at a time. The opposite of lazy is eager: lazy evaluation and eager evaluation are actual technical terms in programming language theory.

Our Sentence implementations so far have not been lazy because the `__init__` eagerly builds a list of all words in the text, binding it to the self.words attribute. This will entail processing the entire text, and the list may use as much memory as the text itself(probably more; it depends on how many non-word characters are in the text). Most of this work will be in vain if the user only iterates over the first couple of words.

The re.finditer function is a lazy version of re.findall which, instead of a list, returns a generator producing re.MatchObject instances on demand. If there are many matches, re.finditer saves a lot of memory. Using it, our third version of Sentence is now lazy: it only produces the next word when it is needed.

In [ ]:
RE_WORD = re.compile("\w+")

class Sentence:

  def __init__(self, text):
    self.text = text

  def __repr__(self):
    return "Sentence(%s)" % reprlib.repr(self.text)

  """
  finditer builds an iterator over the matches of RE_WORD on self.text, yielding MatchObject instances.
  """
  def __iter__(self):
    # match.group() extracts the actual matched text from the MatchObject instance.
    for match in RE_WORD.finditer(self.text):
      yield match.group()   # Yield the current word.
    return

## Sentence take #5: a generator expression


A generator expression can be understood as a lazy version of a list comprehension: it does not eagerly build a list, but returns a generator that will lazily produce the items on demand. 

In other words, if a list comprehension is a factory of lists, a generator expression is a factory of generators.

In [ ]:
# this generator function is used by a list comprehension, then by a generator expression.
def gen_ab():
  print("Start")
  yield "A"
  print("Continue")
  yield "B"
  print("end.")

In [ ]:
# The list comprehension eagerly iterates over the items yielded by the generator object
res1 = [x*3 for x in gen_ab()]

Start
Continue
end.


In [ ]:
for i in res1:
  print("-->", i)

--> AAA
--> BBB


In [ ]:
# The generator expression returns res2. but that call returns a generator which is not consumed here.
res2 = (x*3 for x in gen_ab())

In [ ]:
res2

<generator object <genexpr> at 0x7f3e2417b938>

In [ ]:
# Only when the for loop iterates over res2, the body of gen_AB actually executes.
for i in res2:
  print("-->", i)

Start
--> AAA
Continue
--> BBB
end.


So, a generator expression produces a generator, and we can use it to further reduce the code in the Sentence class.

In [ ]:
RE_WORD = re.compile("\w+")

class Sentence:

  def __init__(self, text):
    self.text = text

  def __repr__(self):
    return "Sentence(%s)" % reprlib.repr(self.text)

  """
  finditer builds an iterator over the matches of RE_WORD on self.text, yielding MatchObject instances.
  """
  def __iter__(self):
    """
    The only difference from is the __iter__ method which here is not a
    generator function (it has no yield) but uses a generator expression to build a generator
    and then returns it. The end result is the same: the caller of __iter__ gets a generator object.
    """
    return (match.group() for match in RE_WORD.finditer(self.text))

### Generator expressions: when to use them

I used several generator expressions when implementing the Vector class.In all those methods, a list comprehension would also work, at the cost of using more memory to store the intermediate list values.

On the other hand, generator functions are much more flexible: you can code complex logic with multiple statements, and can even use them as coroutines.

The rule of thumb in choosing the syntax to use is simple: if the generator expression spans more than a couple of lines, I prefer to code a generator function for the sake of readability. Also, because generator functions have a name, they can be reused. You can always name a generator expression and use it later by assigning it to a variable, of course, but that is stretching its intended usage as a one-off generator.



### Another example: arithmetic progression generator

The classic Iterator pattern is all about traversal: navigating some data structure. But a standard interface based on a method to fetch the next item in a series is also useful when the items are produced on the fly, instead of retrieved from a collection. 

For example, the range built-in generates a bounded arithmetic progression (AP) of integers, and the itertools.count function generates a boundless AP.

In [ ]:
class ArithmeticProgression:

  def __init__(self, begin, step, end=None):
    self.begin = begin
    self.step = step
    self.end = end     # None -> "infinite" series

  def __iter__(self):
    # produces a result value equal to self.begin, but coerced to the type of the subsequent additions
    result = type(self.begin + self.step) ( self.begin)
    forever = self.end is None
    index = 0
    while forever or result < self.end:
      yield result
      index += 1
      # The next potential result is calculated. It may never be yielded, because the while loop may terminate.
      result = self.begin + self.step * index

In [ ]:
# Demonstration of an ArithmeticProgression class
ap = ArithmeticProgression(0, 1, 3)
list(ap)

[0, 1, 2]

In [ ]:
ap = ArithmeticProgression(0, .5, 3)
list(ap)

[0.0, 0.5, 1.0, 1.5, 2.0, 2.5]

In [ ]:
ap = ArithmeticProgression(0, 1/3, 1)
list(ap)

[0.0, 0.3333333333333333, 0.6666666666666666]

In [ ]:
from fractions import Fraction
from decimal import Decimal

In [ ]:
ap = ArithmeticProgression(0, Fraction(1, 3), 1)
list(ap)

[Fraction(0, 1), Fraction(1, 3), Fraction(2, 3)]

In [ ]:
ap = ArithmeticProgression(0, Decimal('.1'), .3)
list(ap)

[Decimal('0'), Decimal('0.1'), Decimal('0.2')]

### Arithmetic progression with itertools

The itertools module in Python 3.4 has 19 generator functions that can be combined in a variety of interesting ways.

In [ ]:
import itertools

In [ ]:
gen = itertools.count(1, .5)

In [ ]:
next(gen)

1

In [ ]:
next(gen)

1.5

In [ ]:
next(gen)

2.0

In [ ]:
next(gen)

2.5

In [ ]:
next(gen)

3.0

However, itertools.count never stops, so if you call list(count()), Python will try to build a list larger than available memory and your machine will be very grumpy long before the call fails.

On the other hand, there is the itertools.takewhile function: it produces a generator which consumes another generator and stops when a given predicate evaluates to False. So we can combine the two and write this:

In [ ]:
gen = itertools.takewhile(lambda n: n < 3, itertools.count(1, .5))
list(gen)

[1, 1.5, 2.0, 2.5]

Leveraging takewhile and count is sweet and short.

In [ ]:
def aritprog_gen(begin, step, end=None):
  first = type(begin + step) (begin)
  ap_gen = itertools.count(first, step) 
  
  if end is not None:
    ap_gen = itertools.takewhile(lambda n: n < end, ap_gen)
  return ap_gen

Note that aritprog_gen is not a generator function: it has no yield in its body. But it returns a generator, so it operates as a generator factory, just as generator function does.

when implementing generators, know what is available in the standard library, otherwise there’s a good chance you’ll reinvent the wheel.

## Generator functions in the standard library

The standard library provides many generators, from plain text file objects providing line by line iteration, to the awesome os.walk function which yields file names while traversing a directory tree, making recursive file system searches as simple as a for loop.

The os.walk generator function is impressive, but I want to focus on general purpose functions that take arbitrary iterables as arguments and return generators that produce selected, computed or rearranged items.

The first group are filtering generator functions: they yield a subset of items produced by the input iterable, without changing the items themselves.

- **dropwhile(predicate, it)**:consumes it skipping items while predicate computes truthy, then yields every remaining item (no further checks are made)
- **filter(predicate, it)**:applies predicate to each item of iterable, yielding the item if predicate(item) is truthy; if predicate is None, only truthy items are yielded
- **filterfalse(predicate, it)**:same as filter, with the predicate logic negated: yields items whenever predicate computes falsy
- **islice(it, stop) or islice(it, start, stop, step=1)**:yields items from a slice of it, similar to `s[:stop]` or `s[start:stop:step]` except it can be any iterable, and the operation is lazy
- **takewhile(predicate, it)**:yields items while predicate computes truthy, then stops and no further checks are made

Like takewhile, most functions take a predicate which is a one-argument boolean function that will be applied to each item in the input to determine whether the item is included in the output.

In [1]:
# Filtering generator functions examples
def vowel(c):
  return c.lower() in "aeiou"

In [2]:
list(filter(vowel, "Aardvark"))

['A', 'a', 'a']

In [3]:
import itertools

list(itertools.filterfalse(vowel, "Aardvark"))

['r', 'd', 'v', 'r', 'k']

In [4]:
list(itertools.dropwhile(vowel, "Aardvark"))

['r', 'd', 'v', 'a', 'r', 'k']

In [5]:
list(itertools.takewhile(vowel, "Aardvark"))

['A', 'a']

In [6]:
list(itertools.compress("Aardvark", (1, 0, 1, 1, 0, 1)))

['A', 'r', 'd', 'a']

In [7]:
list(itertools.islice("Aardvark", 4))

['A', 'a', 'r', 'd']

In [8]:
list(itertools.islice("Aardvark", 4, 7))

['v', 'a', 'r']

In [9]:
list(itertools.islice("Aardvark", 1, 7, 2))

['a', 'd', 'a']

The next group are the mapping generators: they yield items computed from each individual item in the input iterable — or iterables, in the case of map and starmap.

- **accumulate(it, [func])**: yields accumulated sums; if func is provided, yields the result of applying it the first pair of items, then to the first result and next item etc.
- **enumerate(iterable, start=0)**: yields 2-tuples of the form (index, item), where index is counted from start, and item is taken from the iterable
- **map(func, it1, [it2, …, itN])**: applies func to each item of it, yielding the result; if N iterables are given, func must take N arguments and the iterables will be consumed in parallel
- **starmap(func, it)**: applies func to each item of it, yielding the result; the input iterable should yield iterable items iit, and func is applied as func(*iit)

The generators yield one result per item in the input iterables. If the input
comes from more than one iterable, the output stops as soon as the first input iterable is exhausted.

In [10]:
# itertools.accumulate generator function examples.
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]

list(itertools.accumulate(sample))

[5, 9, 11, 19, 26, 32, 35, 35, 44, 45]

In [11]:
list(itertools.accumulate(sample, min))

[5, 4, 2, 2, 2, 2, 2, 0, 0, 0]

In [12]:
list(itertools.accumulate(sample, max))

[5, 5, 5, 8, 8, 8, 8, 8, 9, 9]

In [13]:
import operator

list(itertools.accumulate(sample, operator.mul))

[5, 20, 40, 320, 2240, 13440, 40320, 0, 0, 0]

In [14]:
list(itertools.accumulate(range(1, 11), operator.mul))

[1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

In [15]:
# Mapping generator function examples
list(enumerate("albatroz", 1))

[(1, 'a'),
 (2, 'l'),
 (3, 'b'),
 (4, 'a'),
 (5, 't'),
 (6, 'r'),
 (7, 'o'),
 (8, 'z')]

In [16]:
# Squares of integers from 0 to 10.
list(map(operator.mul, range(11), range(11)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [17]:
# Multiplying numbers from two iterables in parallel: results stop when the shortest iterable ends.
list(map(operator.mul, range(11), [2, 4, 8]))

[0, 4, 16]

In [18]:
# This is what the zip built-in function does.
list(map(lambda a, b: (a, b), range(11), [2, 4, 8]))

[(0, 2), (1, 4), (2, 8)]

In [19]:
# Repeat each letter in the word according to its place in it, starting from 1.
list(itertools.starmap(operator.mul, enumerate("albatroz", 1)))

['a', 'll', 'bbb', 'aaaa', 'ttttt', 'rrrrrr', 'ooooooo', 'zzzzzzzz']

In [20]:
# Running average.
list(itertools.starmap(lambda a, b: (a, b), enumerate(itertools.accumulate(sample), 1)))

[(1, 5),
 (2, 9),
 (3, 11),
 (4, 19),
 (5, 26),
 (6, 32),
 (7, 35),
 (8, 35),
 (9, 44),
 (10, 45)]

Next is the group of merging generators — all of these yield items from multiple input iterables. The chain and chain.from_iterable consume the input iterables sequentially (one after the other), while product, zip and zip_longest consume the input iterables in parallel.

- **chain(it1, …, itN)**: yield all items from it1, then from it2 etc., seamlessly
- **chain.from_iterable(it)**: yield all items from each iterable produced by it, one after the other, seamlessly; it should yield iterable items, for example, a list of iterables
- **product(it1, …, itN, repeat=1)**:cartesian product: yields N-tuples made by combining items from each input iterable like nested for loops could produce; repeat allows the input iterables to be consumed more than once
- **zip(it1, …, itN)**: yields N-tuples built from items taken from the iterables in parallel, silently stopping when the first iterable is exhausted
- **zip_longest(it1, …,itN, fillvalue=None)**: yields N-tuples built from items taken from the iterables in parallel, stopping only when the last iterable is exhausted, filling the blanks with the fill value


In [22]:
# chain is usually called with two or more iterables.
list(itertools.chain("ABC", range(4)))

['A', 'B', 'C', 0, 1, 2, 3]

In [23]:
# chain does nothing useful when called with a single iterable.
list(itertools.chain(enumerate("ABC")))

[(0, 'A'), (1, 'B'), (2, 'C')]

In [24]:
# But chain.from_iterable takes each item from the iterable, and chains them in sequence, as long as each item is itself iterable.
list(itertools.chain.from_iterable(enumerate("ABC")))

[0, 'A', 1, 'B', 2, 'C']

In [25]:
# zip is commonly used to merge two iterables into a series of two-tuples.
list(zip("ABC", range(5)))

[('A', 0), ('B', 1), ('C', 2)]

In [26]:
# Any number of iterables can be consumed by zip in parallel, but the generator stops as soon as the first iterable ends.
list(zip("ABC", range(5), [10, 20, 30, 40]))

[('A', 0, 10), ('B', 1, 20), ('C', 2, 30)]

In [27]:
# works like zip, except it consumes all input iterables to the end, padding output tuples with None as needed.
list(itertools.zip_longest("ABC", range(5)))

[('A', 0), ('B', 1), ('C', 2), (None, 3), (None, 4)]

In [28]:
# The fillvalue keyword argument specifies a custom padding value.
list(itertools.zip_longest("ABC", range(5), fillvalue="?"))

[('A', 0), ('B', 1), ('C', 2), ('?', 3), ('?', 4)]

The itertools.product generator is a lazy way of computing cartesian products, which we built using list comprehensions with more than one for clause. Generator expressions with multiple for clauses can also be used to produce cartesian products lazily.